In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers
import json
import numpy as np
print(tf.__version__)

2.0.0


In [2]:
import sys
print(sys.executable)

/Users/knowledge/Developer/Chatbot/.venv/bin/python


## Hyperparameters

In [3]:
oov_tok = '<OOV>'
num_epochs = 15

## Load Intents

In [4]:
with open('./intents.json', 'r') as f:
    intents = json.load(f)

## Gather and Tokenize Intents, Labels to One-Hot presentation

In [5]:
intent_list = []
label_list = []
for index, intent in enumerate(intents):
    intent_list += intent['patterns']
    num_patterns = len(intent['patterns'])
    label_list += [index] * num_patterns

tokenizer = Tokenizer(oov_token = oov_tok)
tokenizer.fit_on_texts(intent_list)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(intent_list)
max_length = len(max(sequences, key=len))
vocab_size = len(word_index)
padded = pad_sequences(sequences)
padded_length = len(padded[0])
labels = tf.keras.utils.to_categorical(label_list)
num_categories = len(labels[0])

## Model

In [6]:
model = tf.keras.Sequential([
    layers.Embedding(vocab_size+1, 16, input_length=padded_length),
    layers.Flatten(),
    layers.Dense(6, activation='relu'),
    layers.Dense(num_categories, activation='softmax')
])
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['acc'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 8, 16)             400       
_________________________________________________________________
flatten (Flatten)            (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 6)                 774       
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 14        
Total params: 1,188
Trainable params: 1,188
Non-trainable params: 0
_________________________________________________________________


In [7]:
history = model.fit(padded, labels, epochs=num_epochs, verbose=2)

Train on 10 samples
Epoch 1/15


10/10 - 1s - loss: 0.6905 - acc: 0.4000


Epoch 2/15
10/10 - 0s - loss: 0.6854 - acc: 0.5000


Epoch 3/15
10/10 - 0s - loss: 0.6810 - acc: 0.5000


Epoch 4/15
10/10 - 0s - loss: 0.6766 - acc: 0.5000


...

Epoch 11/15
10/10 - 0s - loss: 0.6510 - acc: 0.8000


Epoch 12/15
10/10 - 0s - loss: 0.6479 - acc: 0.9000


Epoch 13/15
10/10 - 0s - loss: 0.6450 - acc: 0.9000


Epoch 14/15
10/10 - 0s - loss: 0.6420 - acc: 0.9000


Epoch 15/15
10/10 - 0s - loss: 0.6390 - acc: 0.9000


In [8]:
sentence = 'i need a loan'
sequence = tokenizer.texts_to_sequences([sentence])
padded_sequence = pad_sequences(sequence, maxlen=padded_length)
prediction = model.predict(padded_sequence)[0]
intent = intents[np.argmax(prediction)]
tag = intent['tag']
print(tag)

Loan


## Export model

In [9]:
model_json = model.to_json()
with open('../lambda/model.json', 'w') as json_file:
    json_file.write(model_json)
model.save_weights('../lambda/model.h5')